In [2]:
!pip install scapy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.9 MB/s eta 0:00:00


In [3]:
from scapy.all import sniff, IP, TCP, UDP

# Define firewall rules
BLOCKED_IPS = ["192.168.1.100", "10.0.0.5"]  # Example blocked IPs
ALLOWED_PORTS = [80, 443, 22]  # HTTP, HTTPS, SSH
BLOCKED_KEYWORDS = ["malware", "hacked", "attack"]

def packet_callback(packet):
    if IP in packet:
        ip_layer = packet[IP]
        src_ip = ip_layer.src
        dst_ip = ip_layer.dst

        # Check if source or destination IP is blocked
        if src_ip in BLOCKED_IPS or dst_ip in BLOCKED_IPS:
            print(f"❌ Blocked packet from {src_ip} to {dst_ip} (Blocked IP)")
            return  # Drop packet

        # Check for allowed ports (TCP/UDP)
        if TCP in packet or UDP in packet:
            port_layer = packet[TCP] if TCP in packet else packet[UDP]
            if port_layer.dport not in ALLOWED_PORTS:
                print(f"❌ Blocked packet from {src_ip} to {dst_ip}:{port_layer.dport} (Unauthorized Port)")
                return  # Drop packet
        if packet.haslayer(TCP) and hasattr(packet[TCP], 'payload'):
            payload = str(packet[TCP].payload)
            if any(keyword in payload for keyword in BLOCKED_KEYWORDS):
                print(f"⚠️ Alert: Malicious content detected in packet from {src_ip} to {dst_ip}")
                return  # Drop packet


        print(f"✅ Allowed packet from {src_ip} to {dst_ip}")

# Start sniffing packets (Runs indefinitely)
print("🚀 Firewall is active. Monitoring network traffic...")
sniff(prn=packet_callback, store=False, count=10)  # Capture 10 packets for testing


🚀 Firewall is active. Monitoring network traffic...
❌ Blocked packet from 172.28.0.1 to 172.28.0.12:6000 (Unauthorized Port)
❌ Blocked packet from 172.28.0.12 to 172.28.0.1:45600 (Unauthorized Port)
❌ Blocked packet from 172.28.0.1 to 172.28.0.12:6000 (Unauthorized Port)
❌ Blocked packet from 172.28.0.12 to 172.28.0.1:45600 (Unauthorized Port)
❌ Blocked packet from 172.28.0.1 to 172.28.0.12:6000 (Unauthorized Port)
❌ Blocked packet from 172.28.0.1 to 172.28.0.12:8080 (Unauthorized Port)
❌ Blocked packet from 172.28.0.12 to 172.28.0.1:36962 (Unauthorized Port)
❌ Blocked packet from 172.28.0.1 to 172.28.0.12:8080 (Unauthorized Port)
❌ Blocked packet from 172.28.0.1 to 172.28.0.12:8080 (Unauthorized Port)
❌ Blocked packet from 172.28.0.12 to 172.28.0.1:36962 (Unauthorized Port)


<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>